前處理 (圖片統一調整成256*256、對訓練資料加強、保存處理後圖片到指定資料夾(processed_data))

In [2]:
import os
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image

# 圖像大小設定
TARGET_SIZE = (256, 256)

# 定義來源與目標資料夾
source_folder = 'data'
output_folder = 'processed_data'

# 創建目標資料夾
os.makedirs(output_folder, exist_ok=True)

# 圖片增強 (只對train資料進行)
augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
])

# 圖片調整函數
def resize_image(image_path, mask=False):
    interpolation = cv2.INTER_NEAREST if mask else cv2.INTER_LINEAR
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, TARGET_SIZE, interpolation=interpolation)
    return image

# 資料處理函數
def process_fold(fold_path, output_fold_path):
    subsets = ['train', 'trainannot', 'val', 'valannot', 'test', 'testannot']

    for subset in subsets:
        input_path = os.path.join(fold_path, subset)
        output_path = os.path.join(output_fold_path, subset)
        os.makedirs(output_path, exist_ok=True)

        for img_name in os.listdir(input_path):
            img_path = os.path.join(input_path, img_name)

            if 'annot' in subset:  # 遮罩
                resized_img = resize_image(img_path, mask=True)
            else:  # 原始影像
                resized_img = resize_image(img_path, mask=False)

                if subset == 'train':  # 對訓練集進行資料增強
                    pil_img = Image.fromarray(resized_img)
                    pil_img = augmentation(pil_img)
                    resized_img = np.array(pil_img)

            save_path = os.path.join(output_path, img_name)
            cv2.imwrite(save_path, resized_img)

# 處理全部5個fold
for fold in ['Fold1', 'Fold2', 'Fold3', 'Fold4', 'Fold5']:
    fold_path = os.path.join(source_folder, fold)
    output_fold_path = os.path.join(output_folder, fold)
    process_fold(fold_path, output_fold_path)

print("資料前處理完成，結果保存在：", output_folder)


資料前處理完成，結果保存在： processed_data
